<img src="https://communications.univie.ac.at/fileadmin/_processed_/csm_Uni_Logo_2016_2f47aacf37.jpg" 
     alt="Logo Universität Wien" 
     width="200"/>

# Practical Machine Learning for Natural Language Processing - 2023 SS  

### Assigment 2 - "Classic" Python   

This assigment is an adaptation for Python of the original exercise ["Unix for Poets"](https://www.cs.upc.edu/~padro/Unixforpoets.pdf)

***

### 1. Generator Functions  

(a) Using [generator functions](https://github.com/rsouza/Python_Course/blob/master/Notebooks/Python_Basic/03_Functions.ipynb), create an object that emulates an eight-sided die (1-8) that is biased, such that the probability of this generator function returning a certain value is proportional to the value itself (for example, the face "6" is 3 times more likely to come out than face "2");  

(b) Using the plt.plot or plt.hist commands, graphically show the result of 10000 plots;  

(c) Modify this generator function so that it terminates automatically when all possible values (1,2,3,4,5,6,7,8) have been returned at least once. In this case, it will return the total absolute time that has occurred since the first iteration.  

In [ ]:
# a)

In [ ]:
# b)

In [ ]:
# c)

In [ ]:
Questão 4 (3.0 pontos)

Crie uma classe chamada "Elevador" que recebe, ao instanciada, o número de andares do edifício e inicia no andar mais baixo.
Crie métodos e propriedades para permitir que o elevador:

    Receba uma chamada (usuário aperta um botão em um andar);
    Receba um andar como destino (usuários apertam botões dentro do elevador);
    Informe em qual andar o elevador está;
    Informe a sequência de andares ainda por visitar;
    Informe a quantidade de vezes que parou em cada andar (passar pelo andar sem "parar" no andar não conta);

Leve em consideração que cada deslocamento de andar demora 3 segundos.
Simule o comportamento do elevador atendendo à dez usuários, que têm andar de origem, andar de destino e sequência de chegada (fora e dentro do elevador) definidos aleatóriamente.


In [ ]:
Questão bônus I (0.5 ponto)

Ilustre graficamente as posições do elevador acima para a simulação pedida:


In [ ]:
Questão bônus II (0.5 ponto)   

Crie um simulador de prédio inteligente, controlando chamadas feitas para n > 1 elevadores:  